In [1]:
import pandas as pd
import random

from google.colab import drive
drive.mount('/content/drive')

train = pd.read_parquet("/content/drive/MyDrive/Colab Notebooks/TP3/parquet/train.parquet")
test = pd.read_parquet("/content/drive/MyDrive/Colab Notebooks/TP3/parquet/test.parquet")

random.seed(666)

Mounted at /content/drive


In [2]:
train.at[18473, 'genre'] = "Children’s Music" #Arreglo una fila bug (género distinto por un caracter al resto).

In [3]:
#SETS DE ENTRENAMIENTO Y VALIDACIÓN

artistas = list(train.artist.unique())
artistas_train = random.sample(artistas, int(len(artistas) * 0.8)) #Divido los unique de artistas en 80% para train.
artistas_validation = list(set(artistas) - set(artistas_train)) #Divido los unique de artistas en 20% para train.

train_set = train[ train['artist'].isin(artistas_train) ] #Genero el set de entrenamiento.
validation_set = train[ train['artist'].isin(artistas_validation) ] #Genero el set de validación

In [4]:
#ELIMINO FILAS DE VALIDATION CON GÉNEROS QUE NO ESTÁN EN TRAIN

generos_train = list(train_set.genre.unique())
validation_set = validation_set[validation_set.genre.isin(generos_train)]

In [5]:
#DROPEO COLUMNAS QUE LEAKEAN INFORMACIÓN

train_set.drop(['artist', 'track_name', 'did', 'a_genres'], axis = 1, inplace = True)
validation_set.drop(['artist', 'track_name', 'did', 'a_genres'], axis = 1, inplace = True)
test.drop(['artist', 'track_name', 'did', 'a_genres'], axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
#Tengo que llenar language (uso el más común) y s-label (promedio)
lenguaje_mas_comun = train_set['language'].value_counts().index[0]
s_label_promedio = train_set['s-label'].mean()

train_set['language'].fillna(lenguaje_mas_comun, inplace = True)
train_set['s-label'].fillna(s_label_promedio, inplace = True)
train_set['lyric'].fillna('', inplace = True)

validation_set['language'].fillna(lenguaje_mas_comun, inplace = True)
validation_set['s-label'].fillna(s_label_promedio, inplace = True)
validation_set['lyric'].fillna('', inplace = True)

test['language'].fillna(lenguaje_mas_comun, inplace = True)
test['s-label'].fillna(s_label_promedio, inplace = True)
test['lyric'].fillna('', inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


# **ENCODING DE VARIABLES CATEGÓRICAS**
#####Time Signature --> Mean Encoding

In [7]:
def MeanEncoding(df, matriz_probas_ts_genero):
  df = pd.merge(df, matriz_probas_ts_genero, on = 'time_signature', how = 'outer')
  y_df = df.loc[:, df.columns == 'genre']
  x_df = df.reset_index().drop(['index', 'genre', 'time_signature'],axis=1)
  return x_df, y_df

ts_por_genero = train_set.groupby(['genre', 'time_signature']).count()[ ['lyric'] ].rename(columns = {'lyric': 'count'}).reset_index(level=['time_signature'])
total_genero = ts_por_genero.groupby(['genre']).sum()

probas_ts_genero = pd.merge(ts_por_genero, total_genero, on = 'genre', how = 'outer')
probas_ts_genero['probas'] = probas_ts_genero['count_x'] / probas_ts_genero['count_y']
probas_ts_genero = probas_ts_genero[ ['time_signature', 'probas'] ].reset_index()

matriz_probas_ts_genero = probas_ts_genero.pivot_table(probas_ts_genero, index = 'time_signature', columns = 'genre').fillna(0)
matriz_probas_ts_genero = matriz_probas_ts_genero['probas'].add_prefix('ts_')

x_validation_set, y_validation_set = MeanEncoding(validation_set, matriz_probas_ts_genero)
x_test_set, y_test_set = MeanEncoding(test, matriz_probas_ts_genero)
x_train_set, y_train_set = MeanEncoding(train_set, matriz_probas_ts_genero)

#####Mode, Key, Language --> One Hot Encoding

In [8]:
from sklearn.preprocessing import OneHotEncoder

def OneHotEncoding(df, rango, features, **kwargs):
  ohe = OneHotEncoder(**kwargs)
  train = df[0]
  valid = df[1]
  test = df[2]
  
  encoded_train = ohe.fit_transform(train[features]).todense().astype(int)
  encoded_valid = ohe.transform(valid[features]).todense().astype(int)
  encoded_test = ohe.transform(test[features]).todense().astype(int)

  cols = dict(zip([x for x in range(rango)], ohe.get_feature_names_out()))

  train = train.reset_index().drop(['index'] + features,axis=1).join(pd.DataFrame(encoded_train).rename(columns = cols))
  valid = valid.reset_index().drop(['index'] + features,axis=1).join(pd.DataFrame(encoded_valid).rename(columns = cols))
  test = test.reset_index().drop(['index'] + features,axis=1).join(pd.DataFrame(encoded_test).rename(columns = cols))

  return train, valid, test
  

In [9]:
#MODE
x_train_set, x_validation_set, x_test_set = OneHotEncoding([x_train_set, x_validation_set, x_test_set], 1, ['mode'], drop = 'first', handle_unknown = 'ignore')

#KEY
x_train_set, x_validation_set, x_test_set = OneHotEncoding([x_train_set, x_validation_set, x_test_set], 12, ['key'], handle_unknown = 'ignore')

#LANGUAGE
x_train_set, x_validation_set, x_test_set = OneHotEncoding([x_train_set, x_validation_set, x_test_set], 13, ['language'], handle_unknown = 'ignore')

#####Lyric --> TFIDF / Count Vectorizer

In [10]:
from sklearn.feature_extraction.text import CountVectorizer#, TfidfVectorizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

#Agrego features por largo de lyric y cantidad de palabras.
x_train_set['lyric_len'] = x_train_set['lyric'].map(lambda x: len(x))
x_train_set['lyric_count'] = x_train_set['lyric'].map(lambda x: len(word_tokenize(x)))

x_validation_set['lyric_len'] = x_validation_set['lyric'].map(lambda x: len(x))
x_validation_set['lyric_count'] = x_validation_set['lyric'].map(lambda x: len(word_tokenize(x)))

x_test_set['lyric_len'] = x_test_set['lyric'].map(lambda x: len(x))
x_test_set['lyric_count'] = x_test_set['lyric'].map(lambda x: len(word_tokenize(x)))

#CountVectorizer
stopwords = set(stopwords.words('english'))
countVec = CountVectorizer(lowercase=True, stop_words=stopwords, max_features=10)
matriz = countVec.fit_transform(x_train_set['lyric'])
palabras_mas_frecuentes = pd.DataFrame(matriz.toarray(), columns=countVec.get_feature_names()).sum().index.tolist()

for palabra in palabras_mas_frecuentes:
  x_train_set["lyric_"+palabra] = x_train_set["lyric"].apply(lambda x: x.count(palabra))
  x_validation_set["lyric_"+palabra] = x_validation_set["lyric"].apply(lambda x: x.count(palabra))
  x_test_set["lyric_"+palabra] = x_test_set["lyric"].apply(lambda x: x.count(palabra))

#Dropeo categoría lyric.
x_train_set = x_train_set.reset_index().drop(['index', 'lyric'],axis=1)
x_validation_set = x_validation_set.reset_index().drop(['index', 'lyric'],axis=1)
x_test_set = x_test_set.reset_index().drop(['index', 'lyric'],axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# **ENTRENAMIENTO DEL MODELO Y MÉTRICAS**
### Búsqueda de hiper-parámetros
### Score para validación y test

In [ ]:
#El score con el set de validación es 0.5017044612420335
#El score con el set de pruebas es 0.494583615436696

In [12]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import top_k_accuracy_score

modelo = xgb.XGBClassifier(random_state = 666)
hiper_parametros = {'n_estimators': [25, 50, 100, 150, 200], 'max_depth': [2, 3, 4, 5, 6], 'learning_rate': [0.05, 0.1, 0.15, 0.2, 0.25], 'subsample': [0.25, 0.5, 0.75, 1, 1.25],
                    'gamma': [0, 0.5, 0.75, 1, 2], 'objective': ['reg:logistic', 'binary:logistic'], 'colsample_bytree': [0.1, 0.25, 0.5, 0.75, 1]}

random_search = RandomizedSearchCV(modelo, hiper_parametros, random_state = 666, n_iter = 30, cv = 3, verbose = 10)
search = random_search.fit(x_train_set, y_train_set)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV 1/3; 1/30] START colsample_bytree=0.75, gamma=1, learning_rate=0.1, max_depth=2, n_estimators=50, objective=reg:logistic, subsample=1.25


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 1/3; 1/30] END colsample_bytree=0.75, gamma=1, learning_rate=0.1, max_depth=2, n_estimators=50, objective=reg:logistic, subsample=1.25;, score=nan total time=   0.4s
[CV 2/3; 1/30] START colsample_bytree=0.75, gamma=1, learning_rate=0.1, max_depth=2, n_estimators=50, objective=reg:logistic, subsample=1.25


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 2/3; 1/30] END colsample_bytree=0.75, gamma=1, learning_rate=0.1, max_depth=2, n_estimators=50, objective=reg:logistic, subsample=1.25;, score=nan total time=   0.3s
[CV 3/3; 1/30] START colsample_bytree=0.75, gamma=1, learning_rate=0.1, max_depth=2, n_estimators=50, objective=reg:logistic, subsample=1.25


...

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 3/3; 29/30] END colsample_bytree=0.5, gamma=2, learning_rate=0.25, max_depth=6, n_estimators=25, objective=binary:logistic, subsample=1;, score=0.438 total time=  21.3s
[CV 1/3; 30/30] START colsample_bytree=1, gamma=0, learning_rate=0.15, max_depth=6, n_estimators=150, objective=binary:logistic, subsample=1.25
[CV 1/3; 30/30] END colsample_bytree=1, gamma=0, learning_rate=0.15, max_depth=6, n_estimators=150, objective=binary:logistic, subsample=1.25;, score=nan total time=   0.2s
[CV 2/3; 30/30] START colsample_bytree=1, gamma=0, learning_rate=0.15, max_depth=6, n_estimators=150, objective=binary:logistic, subsample=1.25


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

[CV 2/3; 30/30] END colsample_bytree=1, gamma=0, learning_rate=0.15, max_depth=6, n_estimators=150, objective=binary:logistic, subsample=1.25;, score=nan total time=   0.2s
[CV 3/3; 30/30] START colsample_bytree=1, gamma=0, learning_rate=0.15, max_depth=6, n_estimators=150, objective=binary:logistic, subsample=1.25
[CV 3/3; 30/30] END colsample_bytree=1, gamma=0, learning_rate=0.15, max_depth=6, n_estimators=150, objective=binary:logistic, subsample=1.25;, score=nan total time=   0.2s


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
33 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
33 fits failed with the followi

In [23]:
search.best_params_

{'subsample': 0.75,
 'objective': 'reg:logistic',
 'n_estimators': 100,
 'max_depth': 4,
 'learning_rate': 0.2,
 'gamma': 0,
 'colsample_bytree': 0.5}

In [24]:
modelo_tuneado = random_search.best_estimator_

proba_preds_valid = modelo_tuneado.predict_proba(x_validation_set)
top_k_accuracy_valid = top_k_accuracy_score(y_validation_set, proba_preds_valid, k = 2, labels = modelo_tuneado.classes_)
print("El score con el set de validación es", top_k_accuracy_valid)

El score con el set de validación es 0.5114865866310953
